In [0]:
ls

sample_data/


#Setup

In [0]:
#TRAINING 1: 5000-0.0001-182555
#TRAINING 2: 10000-0.00001-182555
#TRAINING 3: 15000-0.000001-182555


l_rate = 0.000001
n_steps=15000


# TRAIN
#_NUM_VALIDATION = 10000
#SPLITS_TO_SIZES = {'train': 182555, 'validation': 10000}


# TEST
_NUM_VALIDATION = 0
SPLITS_TO_SIZES = {'train': 192555, 'validation': 0}


In [0]:
#DOWNLOAD MODEL FROM DRIVE IF NEEDED FOR TRAIN OR DETECT

%cd

!pip install -q PyDrive

modelFileId = '1pxrrcTEijmivHPKTBilWZLkD9pmDKLFz'
modelFilePath = 'model.zip'
train_dir = '/tmp/inception_finetuned/'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': modelFileId})
downloaded.GetContentFile(modelFilePath)
print('Files downloaded')

ds = ZipFile(modelFilePath)
ds.extractall(train_dir)
print('Files extracted')

/root
Files downloaded
Files extracted


In [0]:
%cd

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
from google.colab import files

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = os.path.expanduser("~/.kaggle/kaggle.json")
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

/root
Download 100%.


In [0]:
%cd

!pip install kaggle
!git clone https://github.com/tensorflow/models/

/root
    100% |████████████████████████████████| 61kB 2.8MB/s 
    100% |████████████████████████████████| 235kB 5.4MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
Cloning into 'models'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 22343 (delta 0), reused 0 (delta 0), pack-reused 22342
Receiving objects: 100% (22343/22343), 559.85 MiB | 27.76 MiB/s, done.
Resolving deltas: 100% (13202/13202), done.
Checking out files: 100% (2829/2829), done.


In [0]:
%cd

!kaggle competitions download -c airbus-ship-detection

!unzip -q -d train train_v2.zip
!unzip -q train_ship_segmentations_v2.csv.zip

!mkdir test
!mkdir test/ship
!mkdir test/noship
!unzip -q -d test/noship test_v2.zip

!rm *.zip

/root
  0% 0.00/274k [00:00<?, ?B/s]
100% 274k/274k [00:00<00:00, 55.9MB/s]
 83% 15.0M/18.0M [00:00<00:00, 33.9MB/s]
100% 18.0M/18.0M [00:00<00:00, 60.1MB/s]
100% 2.12G/2.12G [00:19<00:00, 148MB/s]

100% 26.4G/26.4G [03:58<00:00, 145MB/s]
100% 26.4G/26.4G [03:58<00:00, 119MB/s]


In [0]:
%cd ~/models/research/slim

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import sys

import tensorflow as tf

from datasets import dataset_utils

# Seed for repeatability.
_RANDOM_SEED = 0

# The number of shards per dataset split.
_NUM_SHARDS = 5

train_dir = '/tmp/inception_finetuned/'
checkpoints_dir = '/tmp/checkpoints'
testdatadir = '/root/test/'


class ImageReader(object):
  """Helper class that provides TensorFlow image coding utilities."""

  def __init__(self):
    # Initializes function that decodes RGB JPEG data.
    self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
    self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

  def read_image_dims(self, sess, image_data):
    image = self.decode_jpeg(sess, image_data)
    return image.shape[0], image.shape[1]

  def decode_jpeg(self, sess, image_data):
    image = sess.run(self._decode_jpeg,
                     feed_dict={self._decode_jpeg_data: image_data})
    assert len(image.shape) == 3
    assert image.shape[2] == 3
    return image


def _get_filenames_and_classes(dataset_dir):
  """Returns a list of filenames and inferred class names.
  Args:
    dataset_dir: A directory containing a set of subdirectories representing
      class names. Each subdirectory should contain PNG or JPG encoded images.
  Returns:
    A list of image file paths, relative to `dataset_dir` and the list of
    subdirectories, representing class names.
  """
  ship_root = dataset_dir
  directories = []
  class_names = []
  for filename in os.listdir(ship_root):
    path = os.path.join(ship_root, filename)
    if os.path.isdir(path):
      directories.append(path)
      class_names.append(filename)

  photo_filenames = []
  for directory in directories:
    for filename in os.listdir(directory):
      path = os.path.join(directory, filename)
      photo_filenames.append(path)

  return photo_filenames, sorted(class_names)


def _get_dataset_filename(dataset_dir, split_name, shard_id):
  output_filename = 'ships_%s_%05d-of-%05d.tfrecord' % (
      split_name, shard_id, _NUM_SHARDS)
  return os.path.join(dataset_dir, output_filename)


def image_to_tfexample(image_data, image_format, height, width, class_id, filename):
  return tf.train.Example(features=tf.train.Features(feature={
      'image/encoded': dataset_utils.bytes_feature(image_data),
      'image/format': dataset_utils.bytes_feature(image_format),
      'image/class/label': dataset_utils.int64_feature(class_id),
      'image/height': dataset_utils.int64_feature(height),
      'image/width': dataset_utils.int64_feature(width),
      'image/filename': dataset_utils.bytes_feature(filename.encode('utf8')),
  }))


def _convert_dataset(split_name, filenames, class_names_to_ids, dataset_dir):
  """Converts the given filenames to a TFRecord dataset.
  Args:
    split_name: The name of the dataset, either 'train' or 'validation'.
    filenames: A list of absolute paths to png or jpg images.
    class_names_to_ids: A dictionary from class names (strings) to ids
      (integers).
    dataset_dir: The directory where the converted datasets are stored.
  """
  assert split_name in ['train', 'validation']

  num_per_shard = int(math.ceil(len(filenames) / float(_NUM_SHARDS)))

  with tf.Graph().as_default():
    image_reader = ImageReader()

    with tf.Session('') as sess:

      for shard_id in range(_NUM_SHARDS):
        output_filename = _get_dataset_filename(
            dataset_dir, split_name, shard_id)

        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
          start_ndx = shard_id * num_per_shard
          end_ndx = min((shard_id+1) * num_per_shard, len(filenames))
          for i in range(start_ndx, end_ndx):
            try:
              sys.stdout.write('\r>> Converting image %d/%d shard %d' % (
                  i+1, len(filenames), shard_id))
              sys.stdout.flush()

              # Read the filename:
              image_data = tf.gfile.FastGFile(filenames[i], 'rb').read()
              height, width = image_reader.read_image_dims(sess, image_data)

              class_name = os.path.basename(os.path.dirname(filenames[i]))
              class_id = class_names_to_ids[class_name]

              example = image_to_tfexample(
                  image_data, b'jpg', height, width, class_id, filenames[i])
              tfrecord_writer.write(example.SerializeToString())
            except:
              print('\nError file:',filenames[i])

  sys.stdout.write('\n')
  sys.stdout.flush()


def _clean_up_temporary_files(dataset_dir):
  """Removes temporary files used to create the dataset.
  Args:
    dataset_dir: The directory where the temporary files are stored.
  """
  filename = _DATA_URL.split('/')[-1]
  filepath = os.path.join(dataset_dir, filename)
  tf.gfile.Remove(filepath)

  tmp_dir = dataset_dir
  tf.gfile.DeleteRecursively(tmp_dir)


def _dataset_exists(dataset_dir):
  for split_name in ['train', 'validation']:
    for shard_id in range(_NUM_SHARDS):
      output_filename = _get_dataset_filename(
          dataset_dir, split_name, shard_id)
      if not tf.gfile.Exists(output_filename):
        return False
  return True


def run_conversion(dataset_dir):
  """Runs the download and conversion operation.
  Args:
    dataset_dir: The dataset directory where the dataset is stored.
  """
  if not tf.gfile.Exists(dataset_dir):
    tf.gfile.MakeDirs(dataset_dir)

  if _dataset_exists(dataset_dir):
    print('Dataset files already exist. Exiting without re-creating them.')
    return

#  dataset_utils.download_and_uncompress_tarball(_DATA_URL, dataset_dir)
  photo_filenames, class_names = _get_filenames_and_classes(dataset_dir)
  class_names_to_ids = dict(zip(class_names, range(len(class_names))))

  # Divide into train and test:
  random.seed(_RANDOM_SEED)
  random.shuffle(photo_filenames)
  training_filenames = photo_filenames[_NUM_VALIDATION:]
  validation_filenames = photo_filenames[:_NUM_VALIDATION]
  with open('validation_filenames.txt', 'w') as f:
    for item in validation_filenames:
        f.write("%s\n" % item)

  # First, convert the training and validation sets.
  _convert_dataset('train', training_filenames, class_names_to_ids,
                   dataset_dir)
  _convert_dataset('validation', validation_filenames, class_names_to_ids,
                   dataset_dir)

  # Finally, write the labels file:
  labels_to_class_names = dict(zip(range(len(class_names)), class_names))
  dataset_utils.write_label_file(labels_to_class_names, dataset_dir)

  #_clean_up_temporary_files(dataset_dir)
  print('\nFinished converting the Ships dataset!')

/root/models/research/slim


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf

from datasets import dataset_utils

slim = tf.contrib.slim

_FILE_PATTERN = 'ships_%s_*.tfrecord'

_NUM_CLASSES = 2

_ITEMS_TO_DESCRIPTIONS = {
    'image': 'A color image of varying size.',
    'label': 'A single integer between 0 and 1',
}


def get_split(split_name, dataset_dir, file_pattern=None, reader=None):
  """Gets a dataset tuple with instructions for reading ships.
  Args:
    split_name: A train/validation split name.
    dataset_dir: The base directory of the dataset sources.
    file_pattern: The file pattern to use when matching the dataset sources.
      It is assumed that the pattern contains a '%s' string so that the split
      name can be inserted.
    reader: The TensorFlow reader type.
  Returns:
    A `Dataset` namedtuple.
  Raises:
    ValueError: if `split_name` is not a valid train/validation split.
  """
  if split_name not in SPLITS_TO_SIZES:
    raise ValueError('split name %s was not recognized.' % split_name)

  if not file_pattern:
    file_pattern = _FILE_PATTERN
  file_pattern = os.path.join(dataset_dir, file_pattern % split_name)

  # Allowing None in the signature so that dataset_factory can use the default.
  if reader is None:
    reader = tf.TFRecordReader

  keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='png'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
      'image/filename': tf.FixedLenFeature((), tf.string, default_value='png'),
  }

  items_to_handlers = {
      'image': slim.tfexample_decoder.Image(),
      'label': slim.tfexample_decoder.Tensor('image/class/label'),
      'filename': slim.tfexample_decoder.Tensor('image/filename'),
  }

  decoder = slim.tfexample_decoder.TFExampleDecoder(
      keys_to_features, items_to_handlers)

  labels_to_names = None
  if dataset_utils.has_labels(dataset_dir):
    labels_to_names = dataset_utils.read_label_file(dataset_dir)

  return slim.dataset.Dataset(
      data_sources=file_pattern,
      reader=reader,
      decoder=decoder,
      num_samples=SPLITS_TO_SIZES[split_name],
      items_to_descriptions=_ITEMS_TO_DESCRIPTIONS,
      num_classes=_NUM_CLASSES,
      labels_to_names=labels_to_names)

In [0]:
from preprocessing import inception_preprocessing
import tensorflow as tf

from tensorflow.contrib import slim


def load_batch(dataset, batch_size=32, height=299, width=299, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label, filename = data_provider.get(['image', 'label', 'filename'])
    
    # Preprocess image for usage by Inception.
    if is_training:
      image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    else:
      image = inception_preprocessing.preprocess_for_eval(image_raw, height, width, central_fraction=1)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels, filenames = tf.train.batch(
          [image, image_raw, label, filename],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size)
    
    return images, images_raw, labels, filenames

In [0]:
%cd

import pandas as pd
import shutil
import glob

os.remove('train/6384c3e78.jpg')

delete_files =  []
for f in delete_files:
        os.remove('/root/test/noship/'+f)

traindatadir = '/root/train/'
df = pd.read_csv('train_ship_segmentations_v2.csv')
print('Move',df['ImageId'].nunique(),'file')

ship_dir = traindatadir + 'ship'
os.mkdir(ship_dir)
dfNotNull = df[df.EncodedPixels.notnull()]
imageIdsShip = dfNotNull['ImageId'].unique()
for file_name in imageIdsShip:
  shutil.move(traindatadir + file_name, ship_dir)
print('Moved in',ship_dir,imageIdsShip.size,'file')

noship_dir = traindatadir + 'noship'
os.mkdir(noship_dir)
for file_name in glob.glob(traindatadir+"/*.jpg"):
  shutil.move(file_name, noship_dir)
print('Moved in',noship_dir)

/root
Move 192556 file
Moved in /root/train/ship 42556 file
Moved in /root/train/noship


#Train

In [0]:
#DOWNLOAD IF YOU DONT NEED TO START FROM MY PRETRAINED MODEL

%cd ~/models/research/slim

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time

from datasets import dataset_utils

# Main slim library
from tensorflow.contrib import slim
from datasets import dataset_utils

url = "http://download.tensorflow.org/models/inception_v4_2016_09_09.tar.gz"

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

/root/models/research/slim
>> Downloading inception_v4_2016_09_09.tar.gz 100.0%
Successfully downloaded inception_v4_2016_09_09.tar.gz 171177982 bytes.


In [0]:
%cd ~/models/research/slim

traindatadir = '/root/train/'
run_conversion(traindatadir)

/root/models/research/slim
>> Converting image 1/182555 shard 0WARNING:tensorflow:From <ipython-input-7-bfc03b39ec18>:123: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
>> Converting image 182555/182555 shard 4
>> Converting image 10000/10000 shard 4

Finished converting the Ships dataset!


In [0]:
%cd ~/models/research/slim

import os

from nets import inception
from preprocessing import inception_preprocessing

from tensorflow.contrib import slim
image_size = inception.inception_v4.default_image_size

batch_size = 32

def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["InceptionV4/Logits", "InceptionV4/AuxLogits"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                break
        else:
            variables_to_restore.append(var)

    return slim.assign_from_checkpoint_fn(
      os.path.join(checkpoints_dir, 'inception_v4.ckpt'),
      variables_to_restore)


with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    dataset = get_split('train', traindatadir)
    images, _, labels, filenames = load_batch(dataset, batch_size = batch_size, height=image_size, width=image_size)

    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v4_arg_scope()):
        logits, _ = inception.inception_v4(images, num_classes=dataset.num_classes, is_training=True)
        
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total_Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=l_rate)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    final_loss = slim.learning.train(
        train_op,
        logdir=train_dir,
        init_fn=get_init_fn(),
        log_every_n_steps=100,
        number_of_steps=n_steps)
        
    print('Finished training. Last batch loss %f' % final_loss)

/root/models/research/slim
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
Instructio

In [0]:
%cd /tmp/inception_finetuned/

!zip -r model.zip model.ckpt-15000* graph.pbtxt checkpoint

#import zipfile
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build


#filenames = glob.glob('model.ckpt-'+str(n_steps)+'*')
#filenames.append('graph.pbtxt')
#filenames.append('checkpoint')

#with zipfile.ZipFile('model.zip', 'w') as zipMe:        
#  for file in filenames:
#    zipMe.write(file, compress_type=zipfile.ZIP_DEFLATED)


auth.authenticate_user()

drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

    media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)

    created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created
  
save_file_to_drive('model.zip', 'model.zip')

/tmp/inception_finetuned
  adding: model.ckpt-15000.data-00000-of-00001 (deflated 7%)
  adding: model.ckpt-15000.index (deflated 72%)
  adding: model.ckpt-15000.meta (deflated 94%)
  adding: graph.pbtxt (deflated 97%)
  adding: checkpoint (deflated 76%)
File ID: 1pxrrcTEijmivHPKTBilWZLkD9pmDKLFz


{'id': '1pxrrcTEijmivHPKTBilWZLkD9pmDKLFz'}

#Detect

In [0]:
# CROP TEST IMAGE IN 4 PARTS

%cd

import os
import shutil
from PIL import Image

noship_path = os.path.join(testdatadir, 'noship')

for f in os.listdir(noship_path):
  f_full = os.path.join(noship_path,f)
  img = Image.open(f_full)
  area1 = (0, 0, 400, 400)
  cropped1 = img.crop(area1)
  cropped1.save(f_full + '_1', 'JPEG')
  area2 = (368, 0, 767, 400)
  cropped2 = img.crop(area2)
  cropped2.save(f_full + '_2', 'JPEG')
  area3 = (0, 368, 400, 767)
  cropped3 = img.crop(area3)
  cropped3.save(f_full + '_3', 'JPEG')
  area4 = (368, 368, 767, 767)
  cropped4 = img.crop(area4)
  cropped4.save(f_full + '_4', 'JPEG')
  
run_conversion(testdatadir)

/root
>> Converting image 1/78030 shard 0WARNING:tensorflow:From <ipython-input-7-bfc03b39ec18>:123: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
>> Converting image 78030/78030 shard 4


Finished converting the Ships dataset!


In [0]:
# CROP VAL IMAGE IN 4 PARTS
# RUN BEFORE run_conversion(traindatadir) IF DIDNT EXECUTED

%cd

import os
import shutil
from PIL import Image

valsplitdatadir = '/root/val_split/'
ship_path = 'val_split/ship/'
noship_path = 'val_split/noship/'
os.mkdir(valsplitdatadir)
os.mkdir(ship_path)
os.mkdir(noship_path)

file_list = []
with open('/root/models/research/slim/validation_filenames.txt') as f:
  file_list = f.read().splitlines()

ship_list = [k for k in file_list if '/ship/' in k]
noship_list = [k for k in file_list if '/noship/' in k]

for f in ship_list:
  shutil.copy(f, ship_path+os.path.basename(f))
for f in noship_list:
  shutil.copy(f, noship_path+os.path.basename(f))
  

for f in os.listdir(ship_path):
  f_full = ship_path + f
  img = Image.open(f_full)
  area1 = (0, 0, 400, 400)
  cropped1 = img.crop(area1)
  cropped1.save(f_full + '_1', 'JPEG')
  area2 = (368, 0, 767, 400)
  cropped2 = img.crop(area2)
  cropped2.save(f_full + '_2', 'JPEG')
  area3 = (0, 368, 400, 767)
  cropped3 = img.crop(area3)
  cropped3.save(f_full + '_3', 'JPEG')
  area4 = (368, 368, 767, 767)
  cropped4 = img.crop(area4)
  cropped4.save(f_full + '_4', 'JPEG')


for f in os.listdir(noship_path):
  f_full = noship_path + f
  img = Image.open(f_full)
  area1 = (0, 0, 400, 400)
  cropped1 = img.crop(area1)
  cropped1.save(f_full + '_1', 'JPEG')
  area2 = (368, 0, 767, 400)
  cropped2 = img.crop(area2)
  cropped2.save(f_full + '_2', 'JPEG')
  area3 = (0, 368, 400, 767)
  cropped3 = img.crop(area3)
  cropped3.save(f_full + '_3', 'JPEG')
  area4 = (368, 368, 767, 767)
  cropped4 = img.crop(area4)
  cropped4.save(f_full + '_4', 'JPEG')

  
run_conversion(valsplitdatadir)

/root
>> Converting image 40000/40000 shard 4
>> Converting image 10000/10000 shard 4

Finished converting the Ships dataset!


In [0]:
%cd ~/models/research/slim

import numpy as np
import tensorflow as tf
from nets import inception
from tensorflow.contrib import slim

loops = 5000

batch_size = 32
image_size = inception.inception_v4.default_image_size

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
#    dataset = get_split('validation', traindatadir)
    dataset = get_split('train', testdatadir)
#    dataset = get_split('train', valsplitdatadir)
    print('dataset.num_samples',dataset.num_samples)
    images, images_raw, labels, filenames = load_batch(dataset, height=image_size, width=image_size, is_training=False)
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v4_arg_scope()):
        logits, _ = inception.inception_v4(images, num_classes=dataset.num_classes)

    probabilities = tf.nn.softmax(logits)
    
    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)
            with open('inference_result.csv', 'w') as f:
                f.write('filename,prediction\n')
                for i in range(loops):
                    sys.stdout.write('\rLoop: '+str(i))
                    sys.stdout.flush()
                    np_probabilities, np_images_raw, np_labels, np_filenames = sess.run([probabilities, images_raw, labels, filenames])
                    for i in range(batch_size): 
                        image = np_images_raw[i, :, :, :]
                        true_label = np_labels[i]
                        predicted_label = np.argmax(np_probabilities[i, :])
                        predicted_name = dataset.labels_to_names[predicted_label]
                        true_name = dataset.labels_to_names[true_label]
                        filename = os.path.basename(np_filenames[i].decode())
                        f.write(filename + ',' + predicted_name+'\n')
#                print('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
#                plt.figure()
#                plt.imshow(image.astype(np.uint8))
#                plt.title('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
#                plt.axis('off')
#                plt.show()

/root/models/research/slim
dataset.num_samples 192555
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.T

In [0]:
%cd ~/models/research/slim

import pandas as pd
from google.colab import files

df = pd.read_csv('inference_result.csv')
inference_unique = df.drop_duplicates()
print('inference_unique shape:',inference_unique.shape)
inference_unique.to_csv('inference_classification.csv', index=False)

#files.download('inference_result.csv')
files.download('inference_classification.csv')

/root/models/research/slim
inference_unique shape: (79749, 2)
